In [6]:
from IPython.display import display

import re
from pathlib import Path

import pandas as pd

from src.processing.taxation_list import _open_file_taxation_list

In [41]:
file_path = Path().absolute().parent / "data/example_taxation_list.docx"

is_import_first_row = False
column_mapping = {
    'number': 0,
    'name': 1,
    'quantity': 2,
    'height': 3,
    'diameter': 4,
    'quality': 5
}
table_data_df = _open_file_taxation_list(file_path, is_import_first_row, column_mapping)
table_data_df

,origin_number,name,quantity,height,diameter,quality
0,1,Чёрная смородина,102 м2,1,-,Хорошее
1,2,Вишня,1,"2,5","0,08",Хорошее
2,3,Крыжовник,8 м2,"0,5",-,Хорошее
3,4,Малина,20 м2,1,-,Хорошее
4,5,Ирга,1,3,-,Хорошее
...,...,...,...,...,...,...
124,125,Яблоня домашняя,1,2,"0,04",Хорошее
125,126,Яблоня домашняя,3 ствола,"1,5х3","0,02х3",Хорошее
126,127,Яблоня домашняя,2 ствола,2х2,"0,05х2",Хорошее
127,128,Малина,1,"0,5",-,Хорошее


In [42]:
filtered_table_data_df = table_data_df[table_data_df['quantity'].str.match(r'^\d+$')].copy(deep=True)
filtered_table_data_df['quantity'] = pd.to_numeric(filtered_table_data_df['quantity'])
filtered_table_data_df = filtered_table_data_df[filtered_table_data_df['quantity'] > 1]
filtered_table_data_df

,origin_number,name,quantity,height,diameter,quality
34,35,Слива,2,"1,5","0,04",Хорошее
48,49,Виноград,2,"1,5х2",-,Хорошее
59,60,Яблоня домашняя,3,1х3,"0,02х3",Хорошее
93,94,Вяз гладкий,10,"1,5х10","0,01х10",Хорошее
109,110,Смородина чёрная,2,"0,5х2",-,Хорошее
115,116,Яблоня домашняя,2,3х3,"0,17; 0,14",Хорошее
117,118,Вишня,2,4х2,"0,20х2",Хорошее


In [43]:
# TODO: Создать нормальный набор данных для тестирования разделения номеров,отдельных деревьев и деревьев со стволами